## Build index and tokenization

In [6]:
import glob
from collections import Counter, defaultdict
import re
import nltk
from bs4 import BeautifulSoup


# tokenize text using nltk lib
def tokenize(text):
    return nltk.word_tokenize(text)


# combines all previous methods together
def preprocess(text):
    return tokenize(text.lower())


def get_deletions(word, n=1):
    dels = set()
    for i in range(len(word)):
        dels.add(word[:i] + word[i + 1:])
    if n > 1:
        tmp = set()
        for w in dels:
            tmp.update(get_deletions(w, n - 1))
        tmp.update(dels)
        return tmp
    return dels


def stem(word):
    ps = nltk.stem.PorterStemmer()
    return ps.stem(word)


def get_reuters_tokens(path='reuters21578/'):
    tokens = []
    for f in glob.glob(path + "reut2*.sgm"):
        reuter_stream = open(f, encoding="latin - 1")
        reuter_content = reuter_stream.read()
        soup = BeautifulSoup(reuter_content, "html.parser")
        articles = soup.find_all("reuters")
        for a in articles:
            title = "" if not a.title else a.title.string
            # id = a.get("newid")
            body = "" if not a.body else a.body.string
            tmp = "\n".join([title, body])
            if len(tmp) < 1:
                continue
            tokens.extend(preprocess(tmp))
    return tokens


def get_big_tokens(path='big.txt'):
    def words(text): return re.findall(r'\w+', text.lower())
    return list(words(open(path).read()))


def build_index(tokens, del_depth=1):
    """
    # principal function - builds an index of terms in all documents
    # returns index: {word: {set_of_orig_words}}
    :param path: path to directory with original reuters files
    """
    index = defaultdict(set)
    freq = Counter()
    for token in tokens:
        freq[token] += 1
        for w in get_deletions(token, del_depth):
            if w in index:
                index[w].add(token)
            else:
                index[w] = {token}
    return index, freq


depth = 2
index, freq = build_index(get_big_tokens(), depth)
print(len(index))

898371


## Distance and correction (of word)

In [8]:
# Actually this is Damerau Levenshtein 
# from: https://www.guyrutenberg.com/2008/12/15/damerau-levenshtein-distance-in-python/
"""
Compute the Damerau-Levenshtein distance between two given
strings (s1 and s2)
"""
def distance(s1, s2):
    xrange = range
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in xrange(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in xrange(-1,lenstr2+1):
        d[(-1,j)] = j+1
 
    for i in xrange(lenstr1):
        for j in xrange(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition
 
    return d[lenstr1-1,lenstr2-1]

distance('poetry', 'peotry')

1

In [15]:
def get_predecessors(words):
    result = set()
    for w in words:
        if w in index:
            result.update(index[w])
    return result
    

# Lets make assumption that depth is 1 or 2 only
def correction_ss(word, depth=2, threshold=0):
    # Depth 0
    if freq[word] > threshold:
        return word
    
    # Depth 1
    # 1. get variants:
    candidates = {}
    # up 1: ~ !!insert!!
    d1 = index[word] # set
    if depth == 2:
        d2 = set(d1) # due to index[word] contains depth 1 and 2
    # down 1:
    dels1 = get_deletions(word, 1) # == !!deletions!!
    d1.update(dels1)
    d1.update(get_predecessors(dels1)) # ~ !!swap!! or !!change!!
#     print(d1)
    result = word
    max_freq = freq[word] 
    if len(d1) > 0:
        for w in d1:
#             print(w, freq[w])
            if freq[w] > max_freq and freq[w] > threshold and distance(word,w) == 1:
                result = w
                max_freq = freq[w]
        if max_freq > threshold:
            return result
        
#     print("go deeper (d2)")
    # Depth 2
    if depth == 2:
        # down 2:
        for w in dels1:
            d2.update(index[w]) # up dels1
        dels2 = get_deletions(word,2).difference(dels1)
        d2.update(dels2)
        for w in dels2:
            d2.update(index[w])
#         print(d2)
        
        result = word
        max_freq = freq[word] 
        for w in d2:
            if freq[w] > max_freq and freq[w] > threshold and distance(word,w) <= 2:
                if distance(word,w) == 1:
                    print("wow!")
                result = w
                max_freq = freq[w]
        if max_freq > threshold:
            return result
    return word 

correction_ss('peotry')

'poetry'

## Norvig's Solution
from here: http://norvig.com/spell-correct.html

In [18]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction_norvig(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Testing

## SymSpell test results

In [24]:
correction = correction_ss

def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    return 'unit_tests pass'

def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print(unit_tests())
spelltest(Testset(open('spell-testset1.txt'))) # Development set
spelltest(Testset(open('spell-testset2.txt'))) # Final test set

unit_tests pass
75% of 270 correct (6% unknown) at 1967 words per second 
68% of 400 correct (11% unknown) at 2814 words per second 


## For Norvig's original solution's unit tests passing

unit_tests pass
75% of 270 correct (6% unknown) at 27 words per second 
68% of 400 correct (11% unknown) at 27 words per second 

In [41]:
def tests(corrector):
    def unit_tests():
        assert corrector('speling') == 'spelling'              # insert
        assert corrector('korrectud') == 'corrected'           # replace 2
        assert corrector('bycycle') == 'bicycle'               # replace
        assert corrector('inconvient') == 'inconvenient'       # insert 2
        assert corrector('arrainged') == 'arranged'            # delete
        assert corrector('peotry') =='poetry'                  # transpose
        assert corrector('peotryy') =='poetry'                 # transpose + delete
        assert corrector('word') == 'word'                     # known
        assert corrector('quintessential') == 'quintessential' # unknown
        return 'unit_tests pass'

    def spelltest(tests, verbose=False):
        "Run corrector(wrong) on all (right, wrong) pairs; report results."
        import time
        start = time.clock()
        good, unknown = 0, 0
        n = len(tests)
        for right, wrong in tests:
            w = corrector(wrong)
            good += (w == right)
            if w != right:
                unknown += (right not in WORDS)
                if verbose:
                    print('corrector({}) => {} ({}); expected {} ({})'
                          .format(wrong, w, WORDS[w], right, WORDS[right]))
        dt = time.clock() - start
        print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
              .format(good / n, n, unknown / n, n / dt))

    def Testset(lines):
        "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
        return [(right, wrong)
                for (right, wrongs) in (line.split(':') for line in lines)
                for wrong in wrongs.split()]

    print(unit_tests())
    spelltest(Testset(open('spell-testset1.txt'))) # Development set
    spelltest(Testset(open('spell-testset2.txt'))) # Final test set

    print('new tests')
    spelltest(Testset(open('aspell.txt')))
    spelltest(Testset(open('wikipedia.txt')))
    spelltest(Testset(open('birkbeck.txt')))

In [42]:
tests(correction_ss)

unit_tests pass
75% of 270 correct (6% unknown) at 426 words per second 
68% of 400 correct (11% unknown) at 1147 words per second 
new tests
43% of 531 correct (23% unknown) at 947 words per second 
61% of 2455 correct (24% unknown) at 698 words per second 
31% of 36133 correct (11% unknown) at 491 words per second 


In [37]:
tests(correction_norvig)

unit_tests pass
75% of 270 correct (6% unknown) at 17 words per second 
68% of 400 correct (11% unknown) at 24 words per second 
new tests
43% of 531 correct (23% unknown) at 12 words per second 
61% of 2455 correct (24% unknown) at 18 words per second 
31% of 36133 correct (11% unknown) at 14 words per second 
